In [ ]:
#EXECUTION
from MinimalTransSim.execution import get_parameters,Write_csv, save_agents
from MinimalTransSim.shanghai_platform_extension import *
from MinimalTransSim.core import get_id
from argparse import Namespace
from importlib import import_module


files_name=['config.py']#file of parameters
args={"output_folder":"data/","no_detail":False,"folder":None}#default values of the optional arguments for the sh-pt.py script

args=Namespace(**args)#to be similar to the sh-pt.py file
configurations=[file[0:-3] for file in files_name]
for config in configurations:#each file of parameters
    try:
        parameter_file=import_module(config.replace('/','.'))#load the parameters, please note the use of import_module from importlib to avoid the top level reference used by __import__
        d=Simu_counter(config)#cretation of the displaying window
        d.start()
        name=args.output_folder#preparation of output files
        name+= config+".csv" if args.folder else "{}_{}.csv".format(get_id(),"general")
        parameter_list,variable_names=get_parameters(parameter_file)
        result_file=Write_csv(name,general_data_to_save+variable_names)
        for PARAMETERS,message in parameter_list():#for each set of parameters (vector mode)
            d.new_sim(message)#update message in displayer
            simu=create_simulation(**PARAMETERS)#creation of the simulation
            simu.set_action(lambda x: d.change(x.time))#connectino with the displaying window
            simu()#execution
            results=extract_result_data(simu)#extraction
            result_file.write(**results,**PARAMETERS)#write in the general file
            if not args.no_detail:
                save_agents(simu,args.output_folder,*particular_files)#create personal file if no_details is not activated
        d.quit()
        d.join()#wait for the displaying window to be closed
    except Exception as e:
        print(type(e).__name__," for the file '",config, "': ",e )#problem?

In [ ]:
#DRAW ANIMATION
dr=Positions_drawing(simu)
dr.set_options(margin=20,real_size=(10,10),time_interval=30,time_coef=1000,repetition=True,hide_axes=True)
dr()

In [ ]:
#PLOT ROUTES
plot_routes(simu)
plot_relative_routes(simu)